# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 10 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.


### Exercise 10.1 

**Use DumpMachine to see how Controlled Z acts on the uniform superposition state
$\right| + + + \cdots +\right> $ .**



In [40]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

operation ReportCZEffect(numQubits : Int) :  Unit {
    using(register = Qubit[numQubits]){
        // Prepares the register in the |+++ ...+〉state
        ApplyToEach(H, register);
        Message("Before the Controlled Z:");
        DumpRegister((),register);
        Controlled Z(Most(register), Tail(register));
        Message("After the Controlled Z:");
        DumpRegister((),register);
        ResetAll(register);
    }
} 

operation ReportCZEffect3() : Unit{
    return ReportCZEffect(3);
}

ReportCZEffect, ReportCZEffect3

In [42]:
%simulate ReportCZEffect3

Before the Controlled Z:


|0⟩	0.35355339059327384 + 0𝑖
|1⟩	0.35355339059327384 + 0𝑖
|2⟩	0.35355339059327384 + 0𝑖
|3⟩	0.35355339059327384 + 0𝑖
|4⟩	0.35355339059327384 + 0𝑖
|5⟩	0.35355339059327384 + 0𝑖
|6⟩	0.35355339059327384 + 0𝑖
|7⟩	0.35355339059327384 + 0𝑖

After the Controlled Z:


|0⟩	0.35355339059327384 + 0𝑖
|1⟩	0.35355339059327384 + 0𝑖
|2⟩	0.35355339059327384 + 0𝑖
|3⟩	0.35355339059327384 + 0𝑖
|4⟩	0.35355339059327384 + 0𝑖
|5⟩	0.35355339059327384 + 0𝑖
|6⟩	0.35355339059327384 + 0𝑖
|7⟩	-0.35355339059327384 + 0𝑖

()

You can see that applying the Controlled Z has flipped the phase of the target or last qubit in the register.

----
### Exercise 10.2

**The operation `ReflectAboutMarkedState` uses `X(flag); H(flag);` to prepare the flag qubit, however you could also have used `H(flag); Z(flag);`.**

**Using either or both of QuTiP and `AssertOperationsEqualReferenced`, prove that these two ways of preparing your flag qubit give you the same reflection.**

In [65]:
open Microsoft.Quantum.Diagnostics;
open GroverSearch;

operation UseXHPreparation(
    markedItemOracle : ((Qubit[], Qubit) => Unit is Adj),
    inputQubits : Qubit[])
: Unit is Adj {
    using (flag = Qubit()) {
        within {
            X(flag);
            H(flag);
        } apply{
            markedItemOracle(inputQubits, flag);                                       
        }
    }
}

operation UseHZPreparation(
    markedItemOracle : ((Qubit[], Qubit) => Unit is Adj),
    inputQubits : Qubit[])
: Unit is Adj {
    using (flag = Qubit()) {
        within {
            H(flag);
            Z(flag);
        } apply{
            markedItemOracle(inputQubits, flag);                                       
        }
    }
}

operation ProveEqualPreparations(): Unit {
    let idxMarkedItem = 6;
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    AssertOperationsEqualReferenced(3,
        UseXHPreparation(markItem, _),
        UseHZPreparation(markItem, _)
    );

    
}

ProveEqualPreparations, UseHZPreparation, UseXHPreparation

In [66]:
%simulate ProveEqualPreparations

()

----
### Exercise 10.3

**Try writing out what `(ControlledOnInt(4, X))(register, flag)` does to the state of `register + [flag]`, using either Dirac notation (check out Chapters 2 and 4 if you need a refresher) or by writing down a unitary matrix that can be used to simulate `(ControlledOnInt(4, X))` acting on a three-qubit register and a flag qubit.
Try doing the same, but for `(ControlledOnInt(4, Z))`.**

> **HINT**: since `(ControlledOnInt(4, X))` acts on four qubits in this example (three control qubits and a target qubit), your unitary matrix should be a stem:[16 \times 16] matrix.

----
### Exercise 10.4

**Try changing the definition of the oracle to control on a different integer.
Does this change the output when you run Grover's algorithm?**

In [56]:
open GroverSearch;
operation RunGroverSearchExercise() : Unit {
    let idxMarkedItem = 2;
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItem(3, markItem);
    Message($"marked {idxMarkedItem} and found {foundItem}.");
    }

RunGroverSearchExercise

In [57]:
%simulate RunGroverSearchExercise

marked 2 and found 2.


()

Chaning the integer it is searching for (also controling on) does not change the output.

----
### Exercise 10.5

**Try modifying your implementation of Grover's algorithm to apply twice as many iterations, and use `DumpMachine` to look at the state that results.
Does that seem like what you'd expect from applying a rotation twice?**

In [85]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;
open GroverSearch;

operation SearchForMarkedItemLonger(
    nQubits : Int,
    markItem : ((Qubit[], Qubit) => Unit is Adj)
)
: Int {
    using (qubits = Qubit[nQubits]) {
        PrepareInitialState(qubits);
        for (idxIteration in 0..NIterations(nQubits) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(PrepareInitialState, qubits);
        }
        Message("After one normal amount of rotation");
        DumpMachine();
        for (idxIteration in 0..NIterations(nQubits) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(PrepareInitialState, qubits);
        }
        Message("After twich the amount of rotation");
        DumpMachine();
        return MeasureInteger(LittleEndian(qubits));
    }
}

operation RunGroverSearchLonger() : Unit {
    let idxMarkedItem = 6;
    let markItem = ApplyOracle(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItemLonger(3, markItem);
    Message($"marked {idxMarkedItem} and found {foundItem}.");
}

RunGroverSearchLonger, SearchForMarkedItemLonger

In [86]:
%simulate RunGroverSearchLonger

After one normal amount of rotation


|0⟩	-0.08838834764831856 + 0𝑖
|1⟩	-0.08838834764831859 + 0𝑖
|2⟩	-0.08838834764831856 + 0𝑖
|3⟩	-0.08838834764831856 + 0𝑖
|4⟩	-0.08838834764831859 + 0𝑖
|5⟩	-0.08838834764831859 + 0𝑖
|6⟩	0.9722718241315038 + 0𝑖
|7⟩	-0.08838834764831857 + 0𝑖

After twich the amount of rotation


|0⟩	-0.375650477505354 + 0𝑖
|1⟩	-0.375650477505354 + 0𝑖
|2⟩	-0.375650477505354 + 0𝑖
|3⟩	-0.375650477505354 + 0𝑖
|4⟩	-0.375650477505354 + 0𝑖
|5⟩	-0.3756504775053541 + 0𝑖
|6⟩	-0.11048543456039839 + 0𝑖
|7⟩	-0.3756504775053542 + 0𝑖

marked 6 and found 4.


()

What you can see here is that if your continue reflecting the state past the ideal number of reflections you get that at twice the optimal number of reflections the marked state is the least likely to be measured (it has the smallest amplitude). This makes some sense as there is a lot of trigonometry involved in finding the optimum number of reflections such that you might see periodic features like this.

----
### Exercise 10.6

**Use `AssertOperationsEqualReferenced` to prove that applying the `T` operation four times does the same thing as applying `Z` once.**

In [97]:
open Microsoft.Quantum.Diagnostics;

operation ProveFourTEqualsOneZ(): Unit{
    AssertOperationsEqualReferenced(1, ApplyToEachA(OperationPowA(T, 4),_), ApplyToEachA(Z,_));
}

FourTEqualsOneZ

In [101]:
%simulate ProveFourTEqualsOneZ

()

**There's another operation `S` that can be thought of as the square root of `Z` (a 90° rotation about the s $Z$-axis); check that applying `T` twice is the same as applying `S` once.**

In [103]:
open Microsoft.Quantum.Diagnostics;

operation ProveTwoTEqualsOneS(): Unit{
    AssertOperationsEqualReferenced(1, ApplyToEachA(OperationPowA(T, 2), _), ApplyToEachA(S, _));
}

TwoTEqualsOneS

In [104]:
%simulate ProveTwoTEqualsOneS

()

----
### Exercise 10.7

**Why don't we need to reset the register of qubits allocated in `EstimateCcnotResources` as shown in Figure 10.10?**

You don't need to reset the register because the `CCNOT` operation didn't do anything. Both control qubits were in the zero state, so that means the operation should do nothing.

----
### Exercise 10.8

**How does the number of `T` operation calls change as you increase the number of control qubits? A rough trend is fine.**

> **HINT:** As you saw above, a controlled-NOT operation with an arbitrary number of qubits can be written as `Controlled X(Most(qs), Tail(qs));`, using functions provided by the `Microsoft.Quantum.Arrays` namespace.

In [50]:
open Microsoft.Quantum.Arrays;

operation EstimateCNotScaling(numQubits : Int) : Unit{
    using(register = Qubit[numQubits]){
        Controlled X(Most(register),Tail(register));
    }
}
operation EstimateCNotScaling4() : Unit{
    return EstimateCNotScaling(4);
}

operation EstimateCNotScaling5() : Unit{
    return EstimateCNotScaling(5);
}

operation EstimateCNotScaling6() : Unit{
    return EstimateCNotScaling(6);
}

operation EstimateCNotScaling7() : Unit{
    return EstimateCNotScaling(7);
}

EstimateCNotScaling, EstimateCNotScaling4, EstimateCNotScaling5, EstimateCNotScaling6, EstimateCNotScaling7

In [51]:
%estimate EstimateCNotScaling4

Metric,Sum
CNOT,30
QubitClifford,6
R,0
Measure,0
T,21
Depth,14
Width,5
BorrowedWidth,0


In [52]:
%estimate EstimateCNotScaling5

Metric,Sum
CNOT,50
QubitClifford,10
R,0
Measure,0
T,35
Depth,23
Width,7
BorrowedWidth,0


In [53]:
%estimate EstimateCNotScaling6

Metric,Sum
CNOT,70
QubitClifford,14
R,0
Measure,0
T,49
Depth,32
Width,9
BorrowedWidth,0


In [54]:
%estimate EstimateCNotScaling7

Metric,Sum
CNOT,90
QubitClifford,18
R,0
Measure,0
T,63
Depth,41
Width,11
BorrowedWidth,0


So the data we see for this correlation is as follows as pairs (# of qubits, # of `T` operations):

((3, 7), (4, 21), (5, 35), (6, 49), (7, 63))

What you can see here is that the more control qubits you have on an operation, the more `T` operations are needed to implement it. The actual relationship is a recursive one that comes from #TODO.


----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [55]:
%version

Component,Version
iqsharp,0.10.2002.2610
Jupyter Core,1.2.36563.0
.NET Runtime,".NETCoreApp,Version=v3.0"
